# Node Postprocessor

Consists of these optional components:
* Reranking
* Filtering
* Contextual compression
* Disambiguation followups
* Query decomposition
* Multi-index retrieval (eg. personalization)

In [1]:
#%pip install --quiet llama-index llama-index-llms-gemini llama-index-embeddings-huggingface pydantic-ai 

In [2]:
MODEL_ID = "gemini-2.0-flash"
EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["HF_TOKEN"][:2] == "hf",\
       "Please specify the HF_TOKEN access token in keys.env file"

In [3]:
import sys
sys.path.append('../basic_rag')
import gutenberg_text_loader as gtl
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [4]:
# for pydantic AI in Jupyter
import nest_asyncio
nest_asyncio.apply()

The examples here are on two Geology texts:
* An 1878 book: The Student's Elements of Geology:  https://www.gutenberg.org/cache/epub/3772/pg3772.txt
* A  1905 book: The Elements of Geology: https://www.gutenberg.org/cache/epub/4204/pg4204.txt

## Plain Semantic Indexing to use as a comparison

In [5]:
#!rm -rf .cache vector_index   # uncomment to start afresh

In [6]:
!ls ./.cache vector_index

./.cache:
pg3772_3736454afe.txt  pg4204_81e8e90db3.txt

vector_index:
default__vector_store.json  graph_store.json	      index_store.json
docstore.json		    image__vector_store.json


In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import Document
import os
import pathlib

INDEX_DIR="vector_index"
Settings.embed_model = HuggingFaceEmbedding(
    model_name=EMBED_MODEL_ID
)

# these are the defaults in LlamaIndex
Settings.chunk_size = 1024; Settings.chunk_overlap = 20; TOP_K=2
#Settings.chunk_size = 100; Settings.chunk_overlap = 10; TOP_K=4

if os.path.isdir(INDEX_DIR):
    print("Loading in already created index")
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_DIR)
    index = load_index_from_storage(storage_context)
else:
    # downloads into .cache the first time
    gs = gtl.GutenbergSource()
    gs.load_from_url("https://www.gutenberg.org/cache/epub/3772/pg3772.txt")
    gs.load_from_url("https://www.gutenberg.org/cache/epub/4204/pg4204.txt")
    # reads all files in .cache
    documents = SimpleDirectoryReader(input_dir="./.cache", required_exts=[".txt"], exclude_hidden=False).load_data()
    # creates a vector db
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=INDEX_DIR)

2025-03-28 20:47:49,298 - INFO - PyTorch version 2.5.1 available.
2025-03-28 20:47:50,858 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-03-28 20:47:59,082 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


Loading in already created index


2025-03-28 20:48:04,343 - INFO - Loading all indices.


In [8]:
from llama_index.llms.gemini import Gemini
from llama_index.core.query_engine import RetrieverQueryEngine

llm = Gemini(model=f"models/{MODEL_ID}", api_key=os.environ["GEMINI_API_KEY"])

def semantic_rag(question, top_k=TOP_K, verbose=True):
    query_engine = RetrieverQueryEngine.from_args(
        retriever=index.as_retriever(similarity_top_k=top_k), llm=llm,
    )
    response = query_engine.query(question)
    response = {
        "answer": str(response),
        "source_nodes": response.source_nodes
    }
    if verbose:
        print(response['answer'])
        for node in response['source_nodes']:
            print(node)
    return response
 
semantic_rag("Describe the geology of the Grand Canyon");

/var/tmp/ipykernel_5749/6092052.py:4: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model=f"models/{MODEL_ID}", api_key=os.environ["GEMINI_API_KEY"])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Grand Canyon's geology consists of stratified rocks, exceeding ten thousand feet in thickness, with a gentle inclination northward. A series of gigantic stairs, often over a thousand feet high and miles broad, rise from the canyon. The retreating escarpments, cliffs of mesas and buttes, and ravine walls are sculpted into architectural forms by weathering and deflation. Wind action dissects plateaus in arid regions, smoothing them into waterless plains of naked rock, residual gravel, or drifting sand.

Node ID: 7b635fb9-7b61-4508-ad6a-370f5cd42822
Text: W. M. DAVIS    HARVARD UNIVERSITY, CAMBRIDGE, MASS.    JULY,
1905            CONTENTS    INTRODUCTION.--THE SCOPE AND AIM OF
GEOLOGY    PART I    EXTERNAL GEOLOGICAL AGENCIES         I. THE WORK
OF THE WEATHER      II. THE WORK OF GROUND WATER     III. RIVERS AND
VALLEYS      IV. RIVER DEPOSITS       V. THE WORK OF GLACIERS      VI.
THE WORK OF ...
Score:  0.771

Node ID: 2e6e56ad-1080-4534-9177-6e25d1db23ff
Text: As they are little 

In [9]:
# shouldn't work because the national forest didn't exist at the time the book was written
semantic_rag("Describe the geology of Petrified National Forest");

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

I am sorry, but the provided text does not contain information about the geology of Petrified National Forest.

Node ID: 7bfb39c5-9900-465d-8f7b-b9f0d49e6617
Text: In the Nova Scotia  field, out of seventy-six distinct coal
seams, twenty are  underlain by old forest grounds.    The presence of
fire clay beneath a seam points in the same  direction. Such
underclays withstand intense heat and are used in  making fire brick,
because their alkalies have been removed by the  long-continued growth
of vegetation....
Score:  0.707

Node ID: a3556434-f9be-45c2-871f-0cf47f859389
Text: — Purity of the Coal explained. — Conversion of Coal into
Anthracite. — Origin of Clay-ironstone. — Marine and brackish-water
Strata in Coal. — Fossil Insects. — Batrachian Reptiles. —
Labyrinthodont Foot-prints in Coal-measures. — Nova Scotia  Coal-
measures with successive Growths of erect fossil Trees. —  Similarity
of American and European...
Score:  0.703



Saved answer:
<pre>
I'm sorry, but the provided text does not contain information about the geology of Petrified National Forest.
<pre>

## Limitations of plain-vanilla Semantic Indexing



In [10]:
semantic_rag("Describe the geology of the Grand Canyon");

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Grand Canyon is north of the high plateaus of northern Arizona and southern Utah. The canyon is cut into stratified rocks that are more than ten thousand feet thick with a gentle inclination northward. From the broad platform rises a series of gigantic stairs, often more than one thousand feet high and a score or more miles in breadth. The retreating escarpments and the walls of the ravines are carved into architectural forms by weathering and deflation.

Node ID: 7b635fb9-7b61-4508-ad6a-370f5cd42822
Text: W. M. DAVIS    HARVARD UNIVERSITY, CAMBRIDGE, MASS.    JULY,
1905            CONTENTS    INTRODUCTION.--THE SCOPE AND AIM OF
GEOLOGY    PART I    EXTERNAL GEOLOGICAL AGENCIES         I. THE WORK
OF THE WEATHER      II. THE WORK OF GROUND WATER     III. RIVERS AND
VALLEYS      IV. RIVER DEPOSITS       V. THE WORK OF GLACIERS      VI.
THE WORK OF ...
Score:  0.771

Node ID: 2e6e56ad-1080-4534-9177-6e25d1db23ff
Text: As they are little protected by talus, which  commonly is
removed 

Saved answer:
<pre>
The Grand Canyon is north of the high plateaus of northern Arizona and southern Utah. The canyon is cut into stratified rocks that are more than ten thousand feet thick with a gentle inclination northward. From the broad platform rises a series of gigantic stairs, often more than one thousand feet high and a score or more miles in breadth. The retreating escarpments and the walls of the ravines are carved into architectural forms by weathering and deflation.
</pre>
The answer is wrong. The answer is about the plateau within which the canyon exists, not about the canyon itself.

## Reranking / Filtering / Contextual Compression

You can use reranking to identify chunks that actually answer the question, and to remove extraneous bits (contextual compression).
Another benefit is that you can look at the chunks and replace obsolete material (filtering)

In [11]:
response = semantic_rag("Describe the geology of the Grand Canyon", top_k=4)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Grand Canyon is north of the high plateaus of northern Arizona and southern Utah. The plateaus are made of stratified rocks that are more than ten thousand feet thick with a gentle inclination northward. From the broad platform where the canyon was cut, a series of gigantic stairs rise, often more than one thousand feet high and a score or more miles in breadth. The retreating escarpments, cliffs of mesas and buttes, and the walls of the ravines are carved into architectural forms by weathering and deflation. The wind helps dissect the great plateaus in arid regions, smoothing them away to waterless plains of naked rock, residual gravel, or drifting residual sand.

Node ID: 7b635fb9-7b61-4508-ad6a-370f5cd42822
Text: W. M. DAVIS    HARVARD UNIVERSITY, CAMBRIDGE, MASS.    JULY,
1905            CONTENTS    INTRODUCTION.--THE SCOPE AND AIM OF
GEOLOGY    PART I    EXTERNAL GEOLOGICAL AGENCIES         I. THE WORK
OF THE WEATHER      II. THE WORK OF GROUND WATER     III. RIVERS AND
VALLEY

Saved answer:
<pre>
The Grand Canyon is north of the high plateaus of northern Arizona and southern Utah. The plateaus are made of stratified rocks that are more than ten thousand feet thick with a gentle inclination northward. From the broad platform where the canyon was cut, a series of gigantic stairs rise, often more than one thousand feet high and a score or more miles in breadth. The retreating escarpments, cliffs of mesas and buttes, and the walls of the ravines are carved into architectural forms by weathering and deflation. The wind helps dissect the great plateaus in arid regions, smoothing them away to waterless plains of naked rock, residual gravel, or drifting residual sand.
</pre>

This is actually wrong. The answer is about the plateau within which the canyon exists, not about the canyon itself.

In [12]:
response['source_nodes'][0].metadata

{'file_path': '/home/jupyter/generative-ai-design-patterns/examples/generation_aware_retrieval/.cache/pg4204_81e8e90db3.txt',
 'file_name': 'pg4204_81e8e90db3.txt',
 'file_type': 'text/plain',
 'file_size': 685928,
 'creation_date': '2025-03-27',
 'last_modified_date': '2025-03-27'}

In [13]:
response['source_nodes'][0].text

"W. M. DAVIS\r\n\r\nHARVARD UNIVERSITY, CAMBRIDGE, MASS.\r\n\r\nJULY, 1905\r\n\r\n\r\n\r\n\r\n\r\nCONTENTS\r\n\r\nINTRODUCTION.--THE SCOPE AND AIM OF GEOLOGY\r\n\r\nPART I\r\n\r\nEXTERNAL GEOLOGICAL AGENCIES\r\n\r\n     I. THE WORK OF THE WEATHER\r\n    II. THE WORK OF GROUND WATER\r\n   III. RIVERS AND VALLEYS\r\n    IV. RIVER DEPOSITS\r\n     V. THE WORK OF GLACIERS\r\n    VI. THE WORK OF THE WIND\r\n   VII. THE SEA AND ITS SHORES\r\n  VIII. OFFSHORE AND DEEP-SEA DEPOSITS\r\n\r\nPART  II\r\n\r\nINTERNAL GEOLOGICAL AGENCIES\r\n\r\n    IX. MOVEMENTS OF THE EARTH'S CRUST\r\n     X. EARTHQUAKES\r\n    XI. VOLCANOES\r\n   XII. UNDERGROUND STRUCTURES OF IGNEOUS ORIGIN\r\n  XIII. METAMORPHISM AND MINERAL VEINS\r\n\r\nPART III\r\n\r\nHISTORICAL GEOLOGY\r\n\r\n   XIV. THE GEOLOGICAL RECORD\r\n    XV. THE PRE-CAMBRIAN SYSTEMS\r\n   XVI. THE CAMBRIAN\r\n  XVII. THE ORDOVICIAN AND SILURIAN\r\n XVIII. THE DEVONIAN\r\n   XIX. THE CARBONIFEROUS\r\n    XX. THE MESOZOIC\r\n   XXI. THE TERTIARY\r\n  X

In [14]:
from dataclasses import dataclass
import pydantic_ai
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai import Agent

model = GeminiModel(MODEL_ID, api_key=os.getenv('GEMINI_API_KEY'))

@dataclass
class Chunk:
    full_text: str
    publication_year: int
    relevant_text: str
    relevance_score: float

def process_node(query, node):
    system_prompt = """
    You will be given a query and some text.
    1. Assign a publication year if it's clear from the text, else say it's the current year
    2. Remove information from the text that is not relevant for answering the question.
    3. Assign a relevance score between 0 and 1 where 1 means that the text answers the question 
    """
    agent = Agent(model, result_type=Chunk, system_prompt=system_prompt)
    chunk = agent.run_sync(f"**Query**: {query}\n **Full Text**: {node.text}").data
    if node.metadata['file_name'].startswith('pg4204'):
        chunk.publication_year = 1905
    else:
        chunk.publication_year = 1878
    return chunk
                              
chunks = [process_node(response['source_nodes'], node) for node in response['source_nodes']]    

2025-03-28 20:48:41,371 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:48:49,284 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:48:55,312 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:49:02,145 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"


In [15]:
chunks[0].relevant_text

"Geology deals with the rocks of the earth's crust. It learns from their composition and structure how the rocks were made and how they have been modified. It ascertains how they have been brought to their present places and wrought to their various topographic forms, such as hills and valleys, plains and mountains. It studies the vestiges which the rocks preserve of ancient organisms which once inhabited our planet. Geology is the history of the earth and its inhabitants, as read in the rocks of the earth's crust. A short search may find in the rock relics of animals, such as the imprints of shells, which lived when it was deposited; and as these are of kinds whose nearest living relatives now have their home in the sea, we infer that it was on the flat sea floor that the sandstone was laid. Its present position hundreds of feet above sea level proves that it has since emerged to form part of the land; while the flatness of the beds shows that the movement was so uniform and gentle as

In [17]:
latest_year = max([chunk.publication_year for chunk in chunks])

In [16]:
def rerank_rag(query, top_k=TOP_K):
    # retrieve matching nodes
    retriever=index.as_retriever(similarity_top_k=top_k*4)  # 4 times as many
    nodes = retriever.retrieve(query)
    
    # shorten text to what's relevant and get a relevance score
    chunks = [process_node(query, node) for node in nodes]
    chunks = sorted(chunks, key=lambda x: x.relevance_score, reverse=True)
    
    # keep only chunks from the latest publication year
    latest_year = max([chunk.publication_year for chunk in chunks])
    chunks = [chunk for chunk in chunks if chunk.publication_year == latest_year]
    
    # limit to best k
    chunks = chunks[:top_k]
    
    system_prompt = """
    Use the information provided in the context to answer the question.
    Limit your answer to what's known based on the provided information.
    """
    agent = Agent(model, result_type=str, system_prompt=system_prompt)
    answer = agent.run_sync(f"**Query**: {query}\n **Context**: {[chunk.relevant_text for chunk in chunks]}\n **Answer**:").data
    
    response = {
        "answer": answer,
        "source_nodes": chunks # [chunk.relevant_text for chunk in chunks]
    }
    return response

rerank_rag("Describe the geology of the Grand Canyon", top_k=2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-28 20:49:35,740 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:49:41,477 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:49:47,093 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:49:52,314 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:49:57,082 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:50:03,089 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-03-28 20:50:08,421 - INFO - HTTP Request:

{'answer': 'The Grand Canyon of the Colorado River is located in the high plateaus of northern Arizona and southern Utah. The canyon walls have been shaped by running water and weathering, with horizontal harder beds forming vertical cliffs and talus slopes concealing weaker strata. The sides of the canyon descend to the river by gigantic steps due to the sapping and retreat of the upper cliffs. The plateaus north of the canyon are composed of stratified rocks more than ten thousand feet thick with a gentle inclination northward. The retreating escarpments and walls of the ravines are carved into architectural forms by weathering and deflation.\n',
 'source_nodes': [Chunk(full_text="Running water has\ngulched the walls and weathering has everywhere attacked and\ndriven them back. The horizontal harder beds stand out in long\nlines of vertical cliffs, often hundreds of feet in height, at\nwhose feet talus slopes conceal the outcrop of the weaker strata.\nAs the upper cliffs have been sa

Saved answer:
    <pre>
The Grand Canyon of the Colorado River is located in the high plateaus of northern Arizona and southern Utah. The canyon walls have been shaped by running water and weathering, with horizontal harder beds forming vertical cliffs and talus slopes concealing weaker strata. The sides of the canyon descend to the river by gigantic steps due to the sapping and retreat of the upper cliffs. The plateaus north of the canyon are composed of stratified rocks more than ten thousand feet thick with a gentle inclination northward. The retreating escarpments and walls of the ravines are carved into architectural forms by weathering and deflation.
    </pre>

The relevant text of the first two chunks:
<pre>
relevant_text='Running water has gulched the walls and weathering has everywhere attacked and driven them back. The horizontal harder beds stand out in long lines of vertical cliffs, often hundreds of feet in height, at whose feet talus slopes conceal the outcrop of the weaker strata. As the upper cliffs have been sapped and driven back by the weather, broad platforms are left at their bases and the sides of the canyon descend to the river by gigantic steps. Far up and down the canyon the eye traces these horizontal layers, like the flutings of an elaborate molding, distinguishing each by its contour as well as by its color and thickness.\n\nThe Grand Canyon of the Colorado is often and rightly cited as an example of the stupendous erosion which may be accomplished by a river. And yet the Colorado is a young stream and its work is no more than well begun. It has not yet wholly reached grade, and the great task of the river and its tributaries--the task of leveling the lofty plateau to a low plain and of transporting it grain by grain to the sea--still lies almost entirely in the future.\n\n'

relevance_score=0.7
</pre>
and
<pre>
relevant_text='The high plateaus of northern Arizona and southern Utah, north of\nthe Grand Canyon of the Colorado River, are composed of stratified\nrocks more than ten thousand feet thick and of very gentle\ninclination northward. From the broad plat form in which the\ncanyon has been cut rises a series of gigantic stairs, which are\noften more than one thousand feet high and a score or more of\nmiles in breadth. The retreating escarpments, the cliffs of the\nmesas and buttes which they have left behind as outliers, and the\nwalls of the ravines are carved into noble architectural forms--\ninto cathedrals, pyramids, amphitheaters, towers, arches, and\ncolonnades--by the processes of weathering aided by deflation. It\nis thus by the help of the action of the wind that great plateaus\nin arid regions are dissected and at last are smoothed away to\nwaterless plains, either composed of naked rock, or strewed with\nresidual gravels, or covered with drifting residual sand.', relevance_score=0.6
</pre>
    
This is all relevant.  Also, usefully, the chunks' relevant text is now more concise.
Finally, the answer is able to distinguish between the geology of the canyon vs. the geology of the surrounding plateau.
so, the answer is correct.

## Disambiguation

In [82]:
@dataclass
class DisambiguationResult:
    is_ambiguous: bool
    ambiguous_term: str
    possibility_1: str
    possibility_2: str

def disambiguate(query, node1, node2):
    system_prompt = """
    You will be given a query and two retrieved passages on which to base the answer to the query.
    Respond by saying whether the two passages are referring to two different entities with the same term.
    For example, the query might be about "Red River", and one passage might be about the
    Red River in Minnesota whereas the other might be about the Red River on the Oklahoma/Texas border.
    If there is no ambiguity between the two passages, return False for is_ambiguous.
    """
    agent = Agent(model, result_type=DisambiguationResult, system_prompt=system_prompt)
    return agent.run_sync(f"**Query**: {query}\n **Passage 1**: {node1.text}\n **Passage 2**: {node2.text}").data

In [83]:
def get_nodes(query):
    response = semantic_rag(query, top_k=10, verbose=False);
    return response

response = get_nodes("Name the characteristics of coal-bearing strata in Newcastle");

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [84]:
for node in response['source_nodes'][1:]:
    result = disambiguate("Name the characteristics of coal-bearing strata in Newcastle", response['source_nodes'][0], node)
    if result.is_ambiguous:
        print(result)

2025-03-27 22:37:42,922 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"


DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='Newcastle in Pennsylvania', possibility_2='Newcastle in England')


2025-03-27 22:37:43,661 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"


DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='Newcastle in Pennsylvania', possibility_2='Newcastle in England')


2025-03-27 22:37:44,391 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"


...

DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='Newcastle in Pennsylvania', possibility_2='Newcastle in England')


2025-03-27 22:37:48,430 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"


DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='Newcastle in Pennsylvania', possibility_2='Newcastle in Nova Scotia and New Brunswick')


2025-03-27 22:37:49,186 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"


DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='a locality in Pennyslvania', possibility_2='north and central England')


Saved result:
<pre>
DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='Newcastle in Pennsylvania', possibility_2='Newcastle in England')
DisambiguationResult(is_ambiguous=True, ambiguous_term='Newcastle', possibility_1='Newcastle in Pennsylvania', possibility_2='Newcastle in Nova Scotia and New Brunswick')
</pre>